# Imports

In [169]:
import os
from dotenv import load_dotenv

import pandas as pd
import numpy as np
import numba
import random

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from catboost import CatBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import ParameterGrid

from datetime import datetime, timedelta
from pandas_market_calendars import get_calendar
import yfinance as yf

from tqdm.notebook import tqdm

from databuilder import build_spread_backtest_dataset

# Fetch Base Strategy Backtest Data + Add Relevant Info

In [ ]:
load_dotenv(dotenv_path='/Users/teymour/Desktop/qnt-projs/Short-Vol-ML/.env')  # replace with your path
polygon_api_key = os.getenv("POLYGON_API_KEY")

calendar = get_calendar("NYSE")
trading_dates = calendar.schedule(start_date="2023-04-20", end_date=datetime.today()).index.strftime("%Y-%m-%d").values

#  Call the function from databuilder.py to generate a DataFrame with all relevant info for base strategy backtesting
base_backtest_df = build_spread_backtest_dataset(dates=trading_dates, ticker='I:SPX', index_ticker="I:VIX1D", 
                                              options_ticker="SPX", trade_time="09:35", move_adjustment=0.5, spread_width=1, polygon_api_key=polygon_api_key)

In [166]:
# trading assumptions and max loss calculations
base_backtest_df['nat_price_cost'] = base_backtest_df['short_bid_price'] - base_backtest_df['long_ask_price']
base_backtest_df['max_nat_price_loss'] = abs(base_backtest_df['short_strike'].iloc[0] - base_backtest_df['long_strike'].iloc[0]) - base_backtest_df['nat_price_cost']
base_backtest_df['mid_price_cost'] = base_backtest_df['short_mid_price'] - base_backtest_df['long_mid_price']
base_backtest_df['max_mid_price_loss'] = abs(base_backtest_df['short_strike'].iloc[0] - base_backtest_df['long_strike'].iloc[0]) - base_backtest_df['mid_price_cost']
base_backtest_df["contracts"] = 1
base_backtest_df["fees"] = base_backtest_df["contracts"] * 0.04

In [167]:
# implied and realized volatility metrics
base_backtest_df['trade_to_close_vol'] = abs((base_backtest_df['underlying_closing_price'] - base_backtest_df['underlying_price_at_trade']) / base_backtest_df['underlying_price_at_trade']) * 100
base_backtest_df['current_day_IV'] = base_backtest_df['vix1d_value'] / np.sqrt(252)
base_backtest_df['current_day_VRP'] = base_backtest_df['current_day_IV'] - base_backtest_df['trade_to_close_vol']

In [171]:
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=("Daily VRPs Scatter Plot", "Histogram of Current Day VRP"),
    row_heights=[0.5, 0.5]
)

# Scatter Plot in the First Row
fig.add_trace(
    go.Scatter(
        x=base_backtest_df.index,  # x-axis as index (or replace with desired x-axis data)
        y=base_backtest_df['current_day_VRP'],
        mode='markers',
        marker=dict(color='blue'),
        name='Current Day VRP'
    ),
    row=1, col=1
)

# Add a red line at y = 0 in the scatter plot
fig.add_hline(
    y=0, line=dict(color='red', width=2), row=1, col=1
)

# Histogram in the Second Row with lighter blue, translucent bars, and defined borders
fig.add_trace(
    go.Histogram(
        x=base_backtest_df['current_day_VRP'],
        marker=dict(color='lightblue', opacity=0.6, line=dict(color='blue', width=1)),
        name='VRP Distribution'
    ),
    row=2, col=1
)

# Update layout with increased height and hide legend
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="VRP",
    xaxis2_title="VRP",
    yaxis2_title="Frequency",
    height=800,  # Increase this value to make the figure taller
    showlegend=False  # Hide the legend
)

# Show combined plot
fig.show()

# Meta-Labeling

### Historical Data and Feature Engineering

In [168]:
# Download historical OHLCV data for the S&P 500 to create features
sp500_ticker = "^GSPC"
underlying_feature_df = yf.download(sp500_ticker, start="2022-01-01", end="2024-10-25", interval="1d")

# preprocess the DataFrame
underlying_feature_df.columns = underlying_feature_df.columns.get_level_values(0)
underlying_feature_df.index = pd.to_datetime(underlying_feature_df.index).date
underlying_feature_df = underlying_feature_df.rename_axis("t", axis="index")
underlying_feature_df = underlying_feature_df.drop(columns=['Close'])
underlying_feature_df.columns.name = None
underlying_feature_df = underlying_feature_df.rename(columns={
    'Open': 'o',
    'High': 'h',
    'Low': 'l',
    'Adj Close': 'c',
    'Volume': 'v'
})

[*********************100%***********************]  1 of 1 completed


In [ ]:
# Features based on open price ('o') only
for days in range(1, 6):
    underlying_feature_df[f'return_{days}d'] = underlying_feature_df['o'].pct_change(periods=days)

for lag in range(1, 6):
    underlying_feature_df[f'lag_{lag}d'] = underlying_feature_df['o'].shift(lag)

for lag in range(3, 6):
    underlying_feature_df[f'serial_corr_{lag}d'] = underlying_feature_df['o'].rolling(window=lag).apply(lambda x: x.autocorr(), raw=False)

ma_windows = [10, 50, 100, 200]
for window in ma_windows:
    underlying_feature_df[f'{window}d_MA'] = underlying_feature_df['o'].rolling(window=window).mean()

underlying_feature_df['MA_crossover_10_50'] = np.where(underlying_feature_df['10d_MA'] > underlying_feature_df['50d_MA'], 1, 0)

underlying_feature_df['price_to_50d_MA'] = underlying_feature_df['o'] / underlying_feature_df['50d_MA']
underlying_feature_df['price_to_200d_MA'] = underlying_feature_df['o'] / underlying_feature_df['200d_MA']

underlying_feature_df['12d_EMA'] = underlying_feature_df['o'].ewm(span=12, adjust=False).mean()
underlying_feature_df['26d_EMA'] = underlying_feature_df['o'].ewm(span=26, adjust=False).mean()
underlying_feature_df['MACD'] = underlying_feature_df['12d_EMA'] - underlying_feature_df['26d_EMA']
underlying_feature_df['MACD_signal'] = underlying_feature_df['MACD'].ewm(span=9, adjust=False).mean()

underlying_feature_df['momentum_5d'] = underlying_feature_df['o'] / underlying_feature_df['o'].shift(5) - 1
underlying_feature_df['momentum_10d'] = underlying_feature_df['o'] / underlying_feature_df['o'].shift(10) - 1

underlying_feature_df['20d_MA'] = underlying_feature_df['o'].rolling(window=20).mean()
underlying_feature_df['20d_stddev'] = underlying_feature_df['o'].rolling(window=20).std()
underlying_feature_df['upper_band'] = underlying_feature_df['20d_MA'] + (underlying_feature_df['20d_stddev'] * 2)
underlying_feature_df['lower_band'] = underlying_feature_df['20d_MA'] - (underlying_feature_df['20d_stddev'] * 2)

vol_windows = [10, 20, 50, 100]
for window in vol_windows:
    underlying_feature_df[f'{window}d_volatility'] = underlying_feature_df['o'].rolling(window=window).std()

underlying_feature_df['volatility_ratio'] = underlying_feature_df['10d_volatility'] / underlying_feature_df['50d_volatility']

In [ ]:
# Features that require end-of-day information (close, high, low)
underlying_feature_df['high_low_range'] = underlying_feature_df['h'] - underlying_feature_df['l']
for window in vol_windows:
    underlying_feature_df[f'{window}d_high_low_vol'] = underlying_feature_df['high_low_range'].rolling(window=window).std()

underlying_feature_df['tr'] = np.maximum(
    (underlying_feature_df['h'] - underlying_feature_df['l']),
    np.maximum(
        abs(underlying_feature_df['h'] - underlying_feature_df['c'].shift(1)),
        abs(underlying_feature_df['l'] - underlying_feature_df['c'].shift(1))
    )
)
underlying_feature_df['14d_ATR'] = underlying_feature_df['tr'].rolling(window=14).mean()

underlying_feature_df['20d_high'] = underlying_feature_df['h'].rolling(window=20).max()
underlying_feature_df['20d_low'] = underlying_feature_df['l'].rolling(window=20).min()

underlying_feature_df['14d_ATRP'] = underlying_feature_df['14d_ATR'] / underlying_feature_df['c'] * 100

def compute_rsi(data, window=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

underlying_feature_df['14d_RSI'] = compute_rsi(underlying_feature_df['c'], window=14)

# Drop intermediate columns used for calculations
underlying_feature_df.drop(['tr', '20d_stddev', '20d_MA'], axis=1, inplace=True)

lagged_feature_list = [
    'high_low_range', '10d_high_low_vol', '20d_high_low_vol', '50d_high_low_vol', '100d_high_low_vol',
    '14d_ATR', '20d_high', '20d_low', '14d_ATRP', '14d_RSI'
]

for f in lagged_feature_list:
    underlying_feature_df[f] = underlying_feature_df[f].shift()
    underlying_feature_df = underlying_feature_df.rename(columns={f: f'prev_{f}'})

underlying_feature_df = underlying_feature_df.dropna()

In [ ]:
# target variable - whether we should have traded on a given day or not
base_backtest_df['avoid_trade'] = np.where(base_backtest_df['net_pnl'] < 0, 1, 0 )

# add the correct targets to the feature DataFrame
aligned_target = base_backtest_df['avoid_trade'].reindex(underlying_feature_df.index)
underlying_feature_df = pd.concat([underlying_feature_df, aligned_target], axis=1)
underlying_feature_df = underlying_feature_df.rename(columns={'avoid_trade': 'target'})

# Store the features as a variable for quick access
X = underlying_feature_df.drop(['o', 'c', 'h', 'l', 'target'], axis=1)

# Drop null target values and null feature values that were created by lags)
underlying_feature_df = underlying_feature_df.dropna()

### Helper Functions

In [ ]:
def group_by_period(df, training_period_length, backtest_period_length):
    """
    Splits a DataFrame into sequential training and backtesting periods for walk-forward training and testing.

    Parameters
    ----------
    df : pd.DataFrame
        The input DataFrame containing the time-series data to be split into training and backtesting periods.
    training_period_length : int
        The number of data points (e.g., days) to include in each training period.
    backtest_period_length : int
        The number of data points to include in each backtesting period.

    Returns
    -------
    tuple
        A tuple containing:
        - training_keys (list of tuples): A list of (start_index, end_index) pairs for each training period.
        - backtest_keys (list of tuples): A list of (start_index, end_index) pairs for each backtesting period.
        - training_period_data_dict (dict): A dictionary where each key is a (start_index, end_index) pair 
          representing a training period, and each value is a DataFrame containing the data for that period.
        - backtest_period_data_dict (dict): A dictionary where each key is a (start_index, end_index) pair 
          representing a backtesting period, and each value is a DataFrame containing the data for that period.
    """
    training_keys = []
    backtest_keys = []
    training_period_data_dict = {}
    backtest_period_data_dict = {}

    current_end_index = len(df)

    while current_end_index - backtest_period_length - training_period_length >= 0:
        backtest_end_index = current_end_index
        backtest_start_index = backtest_end_index - backtest_period_length
        training_end_index = backtest_start_index
        training_start_index = training_end_index - training_period_length

        if training_start_index < 0:
            break

        training_df = df.iloc[training_start_index:training_end_index]
        backtest_df = df.iloc[backtest_start_index:backtest_end_index]

        training_keys.append((training_start_index, training_end_index))
        backtest_keys.append((backtest_start_index, backtest_end_index))

        training_period_data_dict[(training_start_index, training_end_index)] = training_df
        backtest_period_data_dict[(backtest_start_index, backtest_end_index)] = backtest_df

        current_end_index = backtest_start_index

    training_keys.reverse()
    backtest_keys.reverse()
    training_period_data_dict = {k: training_period_data_dict[k] for k in reversed(training_period_data_dict)}
    backtest_period_data_dict = {k: backtest_period_data_dict[k] for k in reversed(backtest_period_data_dict)}
    
    return training_keys, backtest_keys, training_period_data_dict, backtest_period_data_dict
def purged_k_fold_generator(data, n_splits, embargo_pct):
    """
    Generates purged K-fold cross-validation splits for time series data as a list of tuples,
    ensuring no data leakage between training and test sets by adding an embargo period.

    Parameters
    ----------
    data : pd.DataFrame
        The dataset to split into training and test sets for each fold, indexed by date.
    n_splits : int
        The number of folds for cross-validation.
    embargo_pct : float
        The percentage of data to "embargo" between training and test sets to prevent data leakage. 
        Typically, a small value like 0.01 (1%) is used.

    Returns
    -------
    folds : list of tuples
        A list of (train_data, test_data) tuples for each fold. Each tuple contains:
        - train_data : pd.DataFrame
            The training set for the current fold.
        - test_data : pd.DataFrame
            The test set for the current fold, with an embargo period before it.
    """
    folds = []
    fold_size = len(data) // n_splits
    for i in range(n_splits):
        train_end = fold_size * i
        test_start = train_end + int(fold_size * embargo_pct)
        test_end = fold_size * (i + 1)

        train_data = data.iloc[:train_end]
        test_data = data.iloc[test_start:test_end]

        if not train_data.empty and not test_data.empty:
            folds.append((train_data, test_data))
    
    return folds

def random_search_with_purged_kfold(data, feature_list, param_grid, max_iter=10, n_splits=3, embargo_pct=0.01, target_column='target'):
    """
    Performs random search for hyperparameter tuning using purged K-fold cross-validation on a 
    CatBoostClassifier. The best parameters are selected based on the highest F1 score.

    Parameters
    ----------
    data : pd.DataFrame
        The dataset containing features and target labels, indexed by date.
    feature_list : list of str
        List of feature names to use in the model.
    param_grid : dict
        Dictionary specifying the parameter grid for random search. Keys are parameter names and values are lists 
        of parameter values to sample from.
    max_iter : int, optional
        Maximum number of parameter combinations to sample for random search. Default is 10.
    n_splits : int, optional
        Number of folds for cross-validation. Default is 3.
    embargo_pct : float, optional
        Percentage of data to embargo between training and test sets for each fold. Default is 0.01 (1%).
    target_column : str, optional
        The name of the target column in the data. Default is 'target'.

    Returns
    -------
    best_params : dict
        Dictionary of the best-performing hyperparameters.
    best_score : float
        The highest average F1 score achieved during cross-validation.
    """
    best_params = None
    best_score = -np.inf

    # Generate all parameter combinations, then randomly sample `max_iter` combinations
    all_params = list(ParameterGrid(param_grid))
    sampled_params = random.sample(all_params, min(max_iter, len(all_params)))

    folds = purged_k_fold_generator(data, n_splits, embargo_pct)

    for params in tqdm(sampled_params, desc="Random Search Progress"):
        scores = []

        for train_df, val_df in folds:
            scaler = StandardScaler()
            train_df.loc[:, feature_list] = scaler.fit_transform(train_df[feature_list]).astype('float64')
            val_df.loc[:, feature_list] = scaler.transform(val_df[feature_list]).astype('float64')

            X_train, y_train = train_df[feature_list], train_df[target_column].values.flatten()
            X_val, y_val = val_df[feature_list], val_df[target_column].values.flatten()

            model = CatBoostClassifier(
                loss_function='Logloss',
                class_weights=params.get('class_weights', None),
                depth=params.get('depth', 6),
                learning_rate=params.get('learning_rate', 0.1),
                thread_count=-1,
                verbose=False
            )

            model.fit(
                X_train, y_train,
                eval_set=(X_val, y_val),
                early_stopping_rounds=50,
                use_best_model=True
            )

            y_pred = (model.predict_proba(X_val)[:, 1] > 0.5).astype(int)
            score = f1_score(y_val, y_pred)
            scores.append(score)

        avg_score = np.mean(scores)
        if avg_score > best_score:
            best_score = avg_score
            best_params = params

    return best_params, best_score

def evaluate_classification(y_true, y_pred):
    """
    Evaluate classification metrics and display results for a binary classification model.

    This function calculates the accuracy, precision, recall, and F1 score for a given set of 
    true and predicted labels. It also prints the confusion matrix and a detailed classification 
    report with precision, recall, F1 score, and support for each class.

    Parameters
    ----------
    y_true : array-like or pd.Series
        The ground truth (actual) labels.
    y_pred : array-like or pd.Series
        The predicted labels by the model.

    Returns
    -------
    None
        A detailed classification report is printed.
    """

    # Display the confusion matrix
    conf_matrix = confusion_matrix(y_true, y_pred)
    print("\nConfusion Matrix:")
    print(conf_matrix)

    # Detailed classification report
    report = classification_report(y_true, y_pred)
    print("\nClassification Report:")
    print(report)

### Training Loop

In [ ]:
def metalabel(data, training_periods, testing_periods, quant_feature_list, cat_feature_list, param_grid):

    data = data[:-1].copy()
    
    best_params, best_score = random_search_with_purged_kfold(data, quant_feature_list, param_grid, max_iter=500)

    print(f"Best parameters found: {best_params} with F1 score: {best_score}")

    keys, backtest_keys, period_data_dict, backtest_period_data_dict = group_by_period(
        data, training_periods, testing_periods
    )

    agg_backtest_df = pd.DataFrame()
    num_iterations = len(keys)

    for i in tqdm(range(num_iterations)):
        model_key = keys[i]
        train_df = period_data_dict[model_key].copy()
        scaler = StandardScaler()
        train_df[quant_feature_list] = scaler.fit_transform(train_df[quant_feature_list])

        all_features = quant_feature_list + cat_feature_list
        split_idx = int(len(train_df) * 0.8)

        X_train = train_df[all_features].iloc[:split_idx]
        y_train = train_df['target'].iloc[:split_idx].values.flatten()
        X_val = train_df[all_features].iloc[split_idx:]
        y_val = train_df['target'].iloc[split_idx:].values.flatten()

        model = CatBoostClassifier(
            loss_function='Logloss',
            eval_metric='F1',
            thread_count=-1,
            **best_params
        )

        model.fit(
            X_train, y_train,
            eval_set=(X_val, y_val),
            early_stopping_rounds=50,
            use_best_model=True,
            plot=False,
            verbose=False
        )

        backtest_key = backtest_keys[i]
        backtest_df = backtest_period_data_dict[backtest_key].copy()
        
        backtest_df[quant_feature_list] = scaler.transform(backtest_df[quant_feature_list])
        test_features = backtest_df[all_features]

        probabilities = model.predict_proba(test_features)[:, 1]
        predictions = (probabilities > .5).astype(int)
        confidence = np.maximum(probabilities, 1 - probabilities)

        prediction_df = pd.DataFrame({
            'predicted_avoid_trade': predictions,
            'prediction_confidence': confidence,
            'raw_probability': probabilities
        }, index=backtest_df.index)

        backtest_df = backtest_df.join(prediction_df)

        agg_backtest_df = pd.concat([agg_backtest_df, backtest_df], axis=0)

    return agg_backtest_df

In [ ]:
param_grid = {
    'class_weights': [[1, 3], [1, 3.5], [1, 4], [1, 4.5], [1, 5]],  # Adjust class imbalance
    'depth': [3, 4, 6, 8, 10],  # Depth of trees
    'learning_rate': [0.0001, 0.005, 0.01, 0.05, 0.1],  # Step size shrinkage
}

metalabeled_backtest_df = metalabel(
    data=underlying_feature_df, 
    training_periods=150, 
    testing_periods=1, 
    quant_feature_list=list(X.columns), 
    cat_feature_list=[],
    param_grid=param_grid
)

### Model Evaluation

In [ ]:
y_true = metalabeled_backtest_df['target']
y_pred = metalabeled_backtest_df['predicted_avoid_trade']
evaluate_classification(y_true, y_pred)

# Compare Backtests

### Helper Functions

In [ ]:
def calculate_pnl(row):
    """
    Calculate the profit and loss (PnL) for the given row of backtest data.

    Parameters
    ----------
    row : pd.Series
        A row of data containing information about the trade taken on a 
        given day

    Returns
    -------
    float
        The gross profit or loss (PnL) for the trade. If the calculated final PnL exceeds the maximum 
        allowable loss, it caps the loss at 'max_mid_price_loss'.
    """
    if row['direction'] == 1:
        settlement = row['underlying_closing_price'] - row['short_strike']
        if settlement > 0:
            settlement = 0
            final_pnl = row['mid_price_cost']
        else:
            final_pnl = settlement + row['mid_price_cost']
            
    elif row['direction'] == 0:
        settlement = row['short_strike'] - row['underlying_closing_price']
        if settlement > 0:
            settlement = 0
            final_pnl = row['mid_price_cost']
        else:
            final_pnl = settlement + row['mid_price_cost']

    gross_pnl = np.maximum(final_pnl, row['max_mid_price_loss'] * -1)
    
    return gross_pnl

def sharpe_ratio(returns, annualize=True, periods_per_year=252, risk_free_rate=0.04):
    """
    Calculate the Sharpe ratio for a series of returns.

    Parameters
    ----------
    returns : pd.Series
        A Pandas Series containing the returns, typically in percentage terms.
    annualize : bool, optional (default=True)
        Whether to annualize the Sharpe ratio.
    periods_per_year : int, optional (default=252)
        The number of trading periods in a year. Defaults to 252 (daily returns).
    risk_free_rate : float, optional (default=0.0)
        The risk-free rate of return. Defaults to 0 for simplicity.

    Returns
    -------
    float
        The calculated Sharpe ratio. If annualized, returns the annualized Sharpe ratio.
    """
    # Include only days where we actually traded:
    returns = returns[returns != 0].copy()
    
    excess_returns = returns - risk_free_rate / periods_per_year

    mean_return = excess_returns.mean()
    std_return = excess_returns.std()

    sharpe_ratio = mean_return / std_return

    if annualize:
        sharpe_ratio *= np.sqrt(periods_per_year)
    
    return sharpe_ratio

### Base Strategy Backtest

In [ ]:
# Align base strategy backtest's days with metalabeled strategy for comparison
# Because we lost some days of evaluation on the metealabeled backtest due to the walk-forward approach
base_strat_backtest_df = base_backtest_df.reindex(metalabeled_backtest_df.index)

px.line(base_strat_backtest_df['net_capital']).show()
print(f"Base strategy Sharpe: {round(sharpe_ratio(returns=base_strat_backtest_df['pct_return']), 2)}")

base_strat_win_rate = len(base_strat_backtest_df[base_strat_backtest_df['net_pnl'] > 0]) / len(base_strat_backtest_df)
base_strat_avg_win = base_strat_backtest_df[base_strat_backtest_df['net_pnl'] > 0]['net_pnl'].mean() * 100
base_strat_avg_loss = abs(base_strat_backtest_df[base_strat_backtest_df['net_pnl'] < 0]['net_pnl'].mean() * 100)

print(f"Base strategy win rate: {round(base_strat_win_rate * 100, 2)}%")
print(f"Base strategy average win: ${round(base_strat_avg_win, 2)}")
print(f"Base strategy average loss: ${round(base_strat_avg_loss, 2)}")
print(f"Base strategy expected value per trade: ${round((base_strat_avg_win * base_strat_win_rate) - (base_strat_avg_loss * (1 - base_strat_win_rate)), 2)}")

### Meta-labeled Strategy Backtest

In [ ]:
# Take the meta-model's trade/no trade reccomendations and add them to the base strategy backtest DataFrame to create a meta-labeled backtest DataFrame.
# We will now only calculate PnL for a given day if the meta-model recommends to trade, setting net PnL to 0, effectively skipping the day otherwise
metalabeled_strat_backtest_df = base_backtest_df.copy()
aligned_preds = metalabeled_backtest_df['predicted_avoid_trade'].reindex(metalabeled_strat_backtest_df.index)
metalabeled_strat_backtest_df['predicted_avoid_trade'] = aligned_preds
metalabeled_strat_backtest_df = metalabeled_strat_backtest_df.dropna()
metalabeled_strat_backtest_df['predicted_avoid_trade'] = metalabeled_strat_backtest_df['predicted_avoid_trade'].astype(int)

metalabeled_strat_backtest_df['gross_pnl'] = metalabeled_strat_backtest_df.apply(calculate_pnl, axis=1)
metalabeled_strat_backtest_df['net_pnl'] = np.where(metalabeled_strat_backtest_df['predicted_avoid_trade'] == 0, metalabeled_strat_backtest_df['gross_pnl'] * metalabeled_strat_backtest_df['contracts'] - metalabeled_strat_backtest_df['fees'], 0)

capital = 3000

metalabeled_strat_backtest_df['net_capital'] = capital + (metalabeled_strat_backtest_df['net_pnl']*100).cumsum()
metalabeled_strat_backtest_df['day_begin_net_capital'] = metalabeled_strat_backtest_df['net_capital'] - (metalabeled_strat_backtest_df['net_pnl']*100)
metalabeled_strat_backtest_df['cumulative_pnl'] = metalabeled_strat_backtest_df['net_pnl'].cumsum()
metalabeled_strat_backtest_df['pct_return'] = (metalabeled_strat_backtest_df['net_pnl']* 100)/ metalabeled_strat_backtest_df['day_begin_net_capital']

# Plot and metrics
px.line(metalabeled_strat_backtest_df['net_capital']).show()
print(f"Meta-labeled strategy Sharpe: {round(sharpe_ratio(returns=metalabeled_strat_backtest_df['pct_return']), 2)}")

metalabeled_strat_win_rate = len(metalabeled_strat_backtest_df[metalabeled_strat_backtest_df['net_pnl'] > 0]) / len(metalabeled_strat_backtest_df[metalabeled_strat_backtest_df['predicted_avoid_trade'] == 0])
metalabeled_strat_avg_win = metalabeled_strat_backtest_df[metalabeled_strat_backtest_df['net_pnl'] > 0]['net_pnl'].mean() * 100
metalabeled_strat_avg_loss = abs(metalabeled_strat_backtest_df[metalabeled_strat_backtest_df['net_pnl'] < 0]['net_pnl'].mean() * 100)

print(f"Meta-labeled strategy win rate: {round(metalabeled_strat_win_rate * 100, 2)}%")
print(f"Meta-labeled strategy average win: ${round(metalabeled_strat_avg_win, 2)}")
print(f"Meta-labeled strategy average loss: ${round(metalabeled_strat_avg_loss, 2)}")
print(f"Meta-labeled strategy expected value per trade: ${round((metalabeled_strat_avg_win * metalabeled_strat_win_rate) - (metalabeled_strat_avg_loss * (1 - metalabeled_strat_win_rate)), 2)}")